## Task 2 Artificial Neuron Network (ANN) in Keras

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

### Discover and visualize data file `train.csv`
1. Read in the data file using pandas

In [ ]:
dataframe = pd.read_csv("../magic04.data")
dataframe

2. Display information of the dataset that is just read in

In [ ]:
dataframe.info()

3. Display important statistic attribute of all features to see whether to drop any not column that has large differences in the values

In [ ]:
dataframe.describe()

4. Print out the correlation of the features and the target to find out their relevance

In [ ]:
dataframe.corr()

### Visualize data
#### Visualize the target class values to see if there is any abnormal distribution

In [ ]:
plt.hist(dataframe['critical_temp'], edgecolor='maroon', facecolor='orangered')
plt.xlabel("Temperature (Celsius)")
plt.ylabel("Number of temperature label")
plt.title("Temperature of for the target class")
plt.show()

#### Visualize the number of elements feature to see if there is any abnormal distribution

In [ ]:
plt.hist(dataframe['number_of_elements'], edgecolor='maroon', facecolor='orangered')
plt.xlabel("Numbers of elements")
plt.ylabel("Frequency")
plt.title("Features numbers of elements data distribution")
plt.show()

#### Checking null value in dataset

In [ ]:
dataframe.isnull().sum()

#### Create a copy of dataset, then split into 2/3 for training and the rest for testing

In [ ]:
df = dataframe.copy()

X = df.drop(columns=['critical_temp'])
y = df['critical_temp']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)

#### Feature Scaling
Since it is a linear regression problem ( which aims to find the best-fitting line that describes the relationship between the independent variables (features) and the dependent variable (target)), scaling all feature and the target class to optimize the algorithm of finding the best fitting line and also help the learning model to be trained faster.
1. Using MinMaxScaler()
2. fit into the object of this scaler training features dataset
3. Using that standard to scale the testing features

In [ ]:
scaler = MinMaxScaler()

scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_train

**Transform the target with same method of fitting the training label to the scaler
and scale the testing target**

In [ ]:
scaler.fit(y_train.values.reshape(-1,1))
y_train = scaler.transform(y_train.values.reshape(-1,1))
y_train = y_train.reshape(-1)
y_train = pd.Series(y_train)
y_test = scaler.transform(y_test.values.reshape(-1,1))
y_test = y_test.reshape(-1)
y_test = pd.Series(y_test)
y_test

Splitting the training data set into two subsets, one for training and one for validation

In [ ]:
x_val = X_train[:3000]
partial_X_train = X_train[3000:]
y_val = y_train[:3000]
partial_y_train = y_train[3000:]
partial_X_train

#### Create a function to build a artificial neuron network using Keras API
1. `build_model()` function takes in for parameters
    * Number of hidden layers : `n_hidden`
    * Number of neurons of each layer : `n_neurons`
    * Regularization number (L1, L2): `a`, `b`
2. Model using the optimizers of `RMSprop` with a `learning_rate` equals 0.0002
3. Model using Mean Squared Error as a loss function

In [ ]:
from keras import regularizers
from keras.applications.densenet import layers
from keras import metrics

optimizer=keras.optimizers.Adam()
def build_model(n_hidden= 3, n_neurons = 32, a = 0.001, b=0.002):
    model = keras.Sequential()
    for layer in range(n_hidden):
        model.add(layers.Dense(n_neurons, activation = 'relu', kernel_regularizer=regularizers.L1L2(l1=a, l2=b)))
    model.add(layers.Dense(1))
    model.compile(optimizer=optimizer, loss=keras.losses.MeanSquaredError(), metrics=[metrics.MeanSquaredError()])
    return model

**Fitting a model training data subset**
* With 60 iterations and a `batch_size` for dataset of 512
* Print out two loss with a validation dataset when fit into model

In [ ]:
model = build_model()
history = model.fit(partial_X_train,
                    partial_y_train,
                    epochs=50,
                    batch_size=512,
                    validation_data=(x_val, y_val))

**Predict a temperature for a testing data using the trained model above and print out the results**
* Inserse the prediction into original scale to be friendlier in reading the data temperature

#### Visualize the two loss during fitting into the model to check the efficiency of the model

In [ ]:
mse = history.history['mean_squared_error']
var = np.std(y)**2
def r_squared(mse, var):
    return (1-mse/var)

for i in range(len(mse)):
    print(r_squared(mse[i], var))

In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, "bo", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['mean_squared_error'])

In [ ]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
predictions

In [ ]:
score = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {score[1]:.4f}')

### Fine tune model
To make the model become more accurate by implementing:
* Fine tune some hyperparameter of number of hidden layers `n_hidden`, numeber of neurons `n_neurons`
* Changing the regularization L1 and L2 to reduce the loss and not become overfitting

In [ ]:
param_distribs = {
    'n_hidden': [2,4,6],
    'n_neurons': [32,64],
    'a': [0.0002, 0.001],
    'b': [0.0002, 0.001]
}

#### Build a model and choose the best hyperparameter

In [ ]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [ ]:
search_cv = GridSearchCV(keras_reg, param_distribs, cv = 3)
search_cv.fit(partial_X_train, partial_y_train, epochs=60,validation_data=(x_val,y_val), \
              callbacks=[keras.callbacks.EarlyStopping(patience=30)])
search_cv.best_params_
search_cv.best_score_

**Print out the best parameters after hypertuning**

In [ ]:
search_cv.best_params_

In [ ]:
best_model = search_cv.best_estimator_.model
best_model.summary()

#### evaluating the model and making predictions after hypertuning

In [ ]:
best_model.evaluate(X_train, y_train)
best_model.predict(X_train)

In [ ]:
best_model.evaluate(np.array(X_test),np.array(y_test))
best_model.predict(X_test)